In [1]:
import re
import json
import importlib
import numpy as np
import pandas as pd
from collections import deque
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [3]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [4]:
encoder_model = load_model('encoder_model.h5')

/home/fei/miniconda3/envs/py3_5/lib/python3.5/site-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
decoder_model = load_model('decoder_model.h5')

/home/fei/miniconda3/envs/py3_5/lib/python3.5/site-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
word2ind, ind2word = utils.load_index_word_map()

In [7]:
n_tokens = len(word2ind)

In [8]:
def get_next_line(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.array(word2ind[' ']).reshape((1, 1))

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    next_line = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        first_n = np.argsort(output_tokens[0, -1, :])[-3:]
        probs = output_tokens[0, -1, first_n] / np.sum(output_tokens[0, -1, first_n])
        sampled_token_index = np.random.choice(first_n, p=probs)
        next_line.append(sampled_token_index)
        if len(next_line) > 100:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.array(sampled_token_index).reshape((1, 1))

        # Update states
        states_value = [h, c]

    return next_line

In [21]:
def make_song(seed_text, lines=100):
    seed_chars = list(seed_text)
    seed_inds = np.array([word2ind[char] for char in seed_chars]).reshape((1, -1))
    next_line = pad_sequences(seed_inds, maxlen=100)
    total_preds = []
    for _ in range(lines):
        next_line = np.array(next_line).reshape((1, -1))
        next_line = get_next_line(next_line)
        total_preds += next_line
    total_text = ''.join([ind2word[str(ind)] for ind in total_preds])
    return seed_text + total_text

In [27]:
song = make_song('Look at her face. It\'s a beautiful face', lines=10)

In [28]:
print(song)

Look at her face. It's a beautiful facelights to stop in yourself  
All the store to say  
That you wants me  
And I will be the strange  
A  
And the shame is trues  
And I'll never still be  
And I'll be a bed  
  
Tell me when that's the 
I'm a stronger with the wind  
And the baby will says that's the body  
To be standin that I don't wno toom baby,  
It's a missed of the bad boy  
  
The world I could stop  
And the world is a mind  
 a mind  
I'm gonna be a man  
And I'm gonna be the baby  
I won't belong the sun  
I would breathe ifavore  
I'm a lots  
  
I want to the while  
  
I can see there to the stars  
  
And a start of li this  
I'ma stays the storm  
I don't know you all the bed  
  
I can't be you  
It's all the star oall your head on the same  
  
The start to my shake thing  
Tell more that I can be a shine  
  
Andy  
I'll not see your friend  
I can say you and I want  
I want to be all that you've gotta get you ou stay the way  
I wish I wanna be a mother  
I wou